# Web-crawler for collecting banks information

### The case

Online payment processing company has a number of banks in different countries they work with. They need to collect a database with full information avaliable on web about each bank. As a source of information <i>wise.com</i> will be used.

For each country from input data we scrape all banks within the country and get the following table fields:
- general swift code
- branch name
- branch address
- branch code
- bank name
- city
- country

Then, tables will be merged by swift to get only those countries, in which the company has banks and accounts.

*The last section of code is devoted to merging banks with swifts from the web with banks, that were given without swifts as input. Unfortunately due to absence of standard names in input data, some part of the work was done manually. Ignore that section. The main focus of the notebook is parsing.

In [1]:
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from random import randint
import time

import nltk
import re

Loading and preparing data

In [13]:
# summary sheet contains swifts
bic_summary = (pd.read_excel('Account_Summary_v3.xlsx', sheet_name='Summary')
               .iloc[:-1, :]
               .replace(['England', 'Kenia'], ['United Kingdom', 'Kenya'])
              )

# where swifts are missing we put "n/a"
bic_summary.loc[:, 'swift_short'] = bic_summary.loc[:, 'swift_short'].fillna('n/a')

# do forward fill for region and country
bic_summary.loc[:, ['Region', 'Bank Country']] = \
    bic_summary.loc[:, ['Region', 'Bank Country']].ffill()

# create a unique country + bank_name key to merge summary with detail
bic_summary['merge_key'] = bic_summary['Bank Country'] + bic_summary['Bank Name']


# detail contains detailed information on bank,
# including separate rows for each cash type and currency account for each bank
bic_detail = (pd.read_excel('Account_Summary_v3.xlsx', sheet_name='Detail')
              .iloc[:-1, :]
              .replace(['England', 'Kenia'], ['United Kingdom', 'Kenya'])
             )

# do forward fill for region, country, bank name and cash type
bic_detail.loc[:, ['Region', 'Bank Country', 'Bank Name', 'Cash Type']] = \
    bic_detail.loc[:, ['Region', 'Bank Country', 'Bank Name', 'Cash Type']].ffill()

# create the same key for merge in detail as well
bic_detail['merge_key'] = bic_detail['Bank Country'] + bic_detail['Bank Name']

In [52]:
bic_summary

,Region,Bank Country,# of banks by country,# of banks running,Bank Name,# of accounts,swift_short,merge_key
0,EMEA,Bulgaria,1.0,1.0,UniCredit Bulbank AD,1,UNCRBGSF,BulgariaUniCredit Bulbank AD
1,EMEA,Croatia,1.0,2.0,Zagrebacka Banka,2,ZABAHR2X,CroatiaZagrebacka Banka
2,EMEA,Czech Republic,9.0,3.0,"Česká spořitelna, a.s.",9,GIBACZPX,"Czech RepublicČeská spořitelna, a.s."
3,EMEA,Czech Republic,NaN,4.0,"Československá obchodní banka, a.s.",3,CEKOCZPP,"Czech RepublicČeskoslovenská obchodní banka, a.s."
4,EMEA,Czech Republic,NaN,5.0,"Fio banka, a.s.",2,FIOBCZPP,"Czech RepublicFio banka, a.s."
...,...,...,...,...,...,...,...,...
105,LATAM,Peru,NaN,106.0,Banco de la Nacion,1,BANCPEPL,PeruBanco de la Nacion
106,LATAM,Spain,2.0,107.0,Banco Santander S.A.,2,BSCHESMM,SpainBanco Santander S.A.
107,LATAM,Spain,NaN,108.0,Pershing LLC,1,PRSHUS33,SpainPershing LLC
108,LATAM,United States,2.0,109.0,"Citibank, N.A.",1,CITIUS33,"United StatesCitibank, N.A."


In [53]:
bic_detail

,Region,Bank Country,# of banks by country,Bank Name,Cash Type,Currency,BIC,# of accounts,merge_key
0,EMEA,Bulgaria,1.0,UniCredit Bulbank AD,Merchant Cash,BGN,UNCRBGSF,1,BulgariaUniCredit Bulbank AD
1,EMEA,Croatia,1.0,Zagrebacka Banka,Merchant Cash,HRK,ZABAHR2X,2,CroatiaZagrebacka Banka
2,EMEA,Czech Republic,9.0,"Česká spořitelna, a.s.",Merchant Cash,CZK,GIBACZPX,9,"Czech RepublicČeská spořitelna, a.s."
3,EMEA,Czech Republic,NaN,"Československá obchodní banka, a.s.",Merchant Cash,CZK,CEKOCZPP,3,"Czech RepublicČeskoslovenská obchodní banka, a.s."
4,EMEA,Czech Republic,NaN,"Fio banka, a.s.",Merchant Cash,CZK,FIOBCZPP,2,"Czech RepublicFio banka, a.s."
...,...,...,...,...,...,...,...,...,...
249,LATAM,Spain,2.0,Banco Santander S.A.,Trade Cash,EUR,NaN,1,SpainBanco Santander S.A.
250,LATAM,Spain,NaN,Banco Santander S.A.,Trade Cash,USD,NaN,1,SpainBanco Santander S.A.
251,LATAM,Spain,NaN,Pershing LLC,Trade Cash,USD,NaN,1,SpainPershing LLC
252,LATAM,United States,2.0,"Citibank, N.A.",Trade Cash,USD,NaN,1,"United StatesCitibank, N.A."


In [14]:
# do left join, so each bank in detail would have a swift
# for now we assume, that swifts for each bank are the same if we are talking about cash types and currency
detail_with_swifts = \
    (bic_detail.merge(bic_summary.loc[:, ['merge_key', 'swift_short']],
                     on='merge_key',
                     how='left')
     .drop('merge_key', axis=1)
    )

# do forward fill to assign swift to each bank account
# detail_with_swifts.loc[:, ['Region', 'Bank Country', 'swift_short']] = \
#     detail_with_swifts.loc[:, ['Region', 'Bank Country', 'swift_short']].ffill()

In [183]:
detail_with_swifts

,Region,Bank Country,# of banks by country,Bank Name,Cash Type,Currency,BIC,# of accounts,swift_short
0,EMEA,Bulgaria,1.0,UniCredit Bulbank AD,Merchant Cash,BGN,UNCRBGSF,1,UNCRBGSF
1,EMEA,Croatia,1.0,Zagrebacka Banka,Merchant Cash,HRK,ZABAHR2X,2,ZABAHR2X
2,EMEA,Czech Republic,9.0,"Česká spořitelna, a.s.",Merchant Cash,CZK,GIBACZPX,9,GIBACZPX
3,EMEA,Czech Republic,NaN,"Československá obchodní banka, a.s.",Merchant Cash,CZK,CEKOCZPP,3,CEKOCZPP
4,EMEA,Czech Republic,NaN,"Fio banka, a.s.",Merchant Cash,CZK,FIOBCZPP,2,FIOBCZPP
...,...,...,...,...,...,...,...,...,...
249,LATAM,Spain,2.0,Banco Santander S.A.,Trade Cash,EUR,NaN,1,BSCHESMM
250,LATAM,Spain,NaN,Banco Santander S.A.,Trade Cash,USD,NaN,1,BSCHESMM
251,LATAM,Spain,NaN,Pershing LLC,Trade Cash,USD,NaN,1,PRSHUS33
252,LATAM,United States,2.0,"Citibank, N.A.",Trade Cash,USD,NaN,1,CITIUS33


## Scraping wise.com by country

### Retrieve proxies

To avoid being blocked while parsing, we'll use IP-rotation. Proxies are retrieved from <i>sslproxies.org</i>

In [50]:
def get_proxies(url_proxy):
    
    html = requests.get(url_proxy).text
    soup = BeautifulSoup(html, 'html.parser')

    return soup.find('textarea').text.split('\n')[3:-1]
    
    
url_proxy = 'https://sslproxies.org/'
proxies = get_proxies(url_proxy)

### Create functions for parsing

Function generates request url for a country and a page given as input

In [68]:
def generate_url(country, page):
    if page == 1:
        return f'https://wise.com/us/swift-codes/countries/{country}'
    return f'https://wise.com/us/swift-codes/countries/{country}?page={page}'

In [69]:
generate_url('czech-republic', 2)

'https://wise.com/us/swift-codes/countries/czech-republic?page=2'

Function gets banks' hrefs avaliable for the country on a specific page

In [70]:
def get_country_page_hrefs(country, page, proxy):
    
    html = requests.get(generate_url(country, page),
                        proxies = {"http://": proxy, "https://": proxy}).text
    soup = BeautifulSoup(html, 'html.parser')
    
    bank_hrefs = []
    
    for row in soup.find_all('tr'):
        
        if row.find('td'):
            bank_hrefs.append(row.find('td').find('a')['href'])
    
    return bank_hrefs



In [71]:
get_country_page_hrefs('czech-republic', 2, proxies[0])

['/us/swift-codes/countries/czech-republic/bank-of-communications-co-ltd-prague-branch-odstepny-zavod',
 '/us/swift-codes/countries/czech-republic/banka-creditas',
 '/us/swift-codes/countries/czech-republic/barigol-management-s-r-o',
 '/us/swift-codes/countries/czech-republic/bnp-paribas-s-a-pobocka-ceska-republika',
 '/us/swift-codes/countries/czech-republic/centralni-depozitar-cennych-papiru-a-s']

Function gets all banks' hrefs for a given country

In [72]:
def get_country_bank_hrefs(country, proxies):
    
    page = 1
    
    if page == 1 or page % 10 == 0:
        proxy = proxies[randint(0, 99)]
    
    store_country = []
    
    while True:
        
        country_hrefs = get_country_page_hrefs(country, page, proxy)
        
        if country_hrefs:
            store_country.extend(country_hrefs)
        else:
            break
        
        page += 1
            
    return pd.DataFrame({'hrefs': store_country})



In [74]:
# get_country_bank_hrefs('czech-republic', proxies)

By iterating through list of unique countries from input xlsx file, we get all banks' hrefs and store them in the dataframe with country specified.

In [213]:
store_hrefs = []

for country in ['-'.join(c.lower().split(' ')) for c in bic_data['Bank Country'].unique()]:
    
    country_hrefs = get_country_bank_hrefs(country, proxies)
    country_hrefs['country'] = country
    
    store_hrefs.append(country_hrefs)
    print('loaded hrefs for {}'.format(country))
    

hrefs_df = pd.concat(store_hrefs)

loaded hrefs for bulgaria
loaded hrefs for croatia
loaded hrefs for czech-republic
loaded hrefs for united-kingdom
loaded hrefs for hungary
loaded hrefs for israel
loaded hrefs for kenya
loaded hrefs for netherlands
loaded hrefs for nigeria
loaded hrefs for poland
loaded hrefs for romania
loaded hrefs for russia
loaded hrefs for slovakia
loaded hrefs for south-africa
loaded hrefs for turkey
loaded hrefs for argentina
loaded hrefs for chile
loaded hrefs for colombia
loaded hrefs for mexico
loaded hrefs for panama
loaded hrefs for peru
loaded hrefs for spain
loaded hrefs for united-states


In [215]:
# write hrefs to csv to save the results
hrefs_df.to_csv('pre_bank_hrefs.csv')

In [75]:
hrefs_df = pd.read_csv('pre_bank_hrefs.csv', index_col=0)

In [76]:
hrefs_df

,hrefs,country
0,/us/swift-codes/countries/bulgaria/allianz-ban...,bulgaria
1,/us/swift-codes/countries/bulgaria/bnp-paribas...,bulgaria
2,/us/swift-codes/countries/bulgaria/borica-ad,bulgaria
3,/us/swift-codes/countries/bulgaria/bulgarian-d...,bulgaria
4,/us/swift-codes/countries/bulgaria/bulgarian-n...,bulgaria
...,...,...
1286,/us/swift-codes/countries/united-states/xerox-...,united-states
1287,/us/swift-codes/countries/united-states/yisrae...,united-states
1288,/us/swift-codes/countries/united-states/zf-aut...,united-states
1289,/us/swift-codes/countries/united-states/zions-...,united-states


For each bank href <i>wise.com</i> has several banks - different branches within the country. The following function retrieves information 

In [52]:
def get_bank_info(country, href, proxy):
    
    request_url = f'https://wise.com{href}'
    
    html = requests.get(request_url,
                        proxies = {"http://": proxy, "https://": proxy}).text
    soup = BeautifulSoup(html, 'html.parser')
    
    cells = soup.find_all('li', attrs = {'class': 'list-group-item'})
    
    swifts, bank_names = [], []
    
    for cell in cells:
        
        raw_swift = cell.find('a')['href']
        swift = raw_swift[raw_swift.find('codes/') + 6:]
        bank_name = cell.find('small').find('span', attrs = {'class': 'd-block'}).text
        
        swifts.append(swift)
        bank_names.append(bank_name)

    df = pd.DataFrame({'swift': swifts, 'name_from_wise': bank_names})
    df['bank_country'] = country
    
    return df
    
    

In [256]:
get_bank_info(hrefs_df.country.iloc[2], hrefs_df.hrefs.iloc[2], proxies[0])

,swift,name_from_wise,bank_country
0,BSBGBGSFXXX,BORICA AD,bulgaria
1,BSBGBGSFIOP,BORICA AD,bulgaria


In [258]:
store_info = []

for idx, (country, href) in enumerate(zip(hrefs_df.country, hrefs_df.hrefs)):
    
    if idx % 10 == 0:
        proxy = proxies[randint(0, 99)]
    
    store_info.append(get_bank_info(country, href, proxy))
    
    if idx % 250 == 0:
        print(f'processed {idx} href(s)')
    
    
banks_info_df = pd.concat(store_info)

processed 0 href(s)
processed 250 href(s)
processed 500 href(s)
processed 750 href(s)
processed 1000 href(s)
processed 1250 href(s)
processed 1500 href(s)
processed 1750 href(s)
processed 2000 href(s)
processed 2250 href(s)
processed 2500 href(s)
processed 2750 href(s)
processed 3000 href(s)
processed 3250 href(s)
processed 3500 href(s)
processed 3750 href(s)


In [291]:
banks_info_df['bank_country'] = \
    banks_info_df.bank_country.apply(lambda x: ' '.join(x.split('-')))

In [292]:
banks_info_df.to_csv('banks_info_data.csv')

In [140]:
banks_info_df = pd.read_csv('banks_info_data.csv', index_col=0).drop_duplicates().reset_index(drop=True)

In [141]:
banks_info_df

,swift,name_from_wise,bank_country
0,BUINBGSFXXX,ALLIANZ BANK BULGARIA AD,bulgaria
1,BNPABGSXXXX,BNP PARIBAS S.A.-SOFIA BRANCH,bulgaria
2,BSBGBGSFXXX,BORICA AD,bulgaria
3,BSBGBGSFIOP,BORICA AD,bulgaria
4,NASBBGSFXXX,BULGARIAN DEVELOPMENT BANK AD,bulgaria
...,...,...,...
5954,ZFNBUS55SLC,ZIONS BANCORPORATION N.A.,united states
5955,ZFNBUS55SEA,ZIONS BANCORPORATION N.A.,united states
5956,ZURIUS44FAR,ZURICH AMERICAN INSURANCE COMPANY,united states
5957,ZURIUS44XXX,ZURICH AMERICAN INSURANCE COMPANY,united states


In [68]:
def get_full_bank_info(swift, proxy):
    
    request_url = f'https://wise.com/us/swift-codes/{swift}'
    
    html = requests.get(request_url,
                        proxies = {"http://": proxy, "https://": proxy}).text
    soup = BeautifulSoup(html, 'html.parser')
    
    row_names, row_values = [], []
    
    if soup.find('table') is None:
        return
#         return pd.DataFrame({'Swift code (8 characters)': [],
#                              'Branch name': [],
#                              'Branch address': [],
#                              'Branch code': [],
#                              ''
#                             })
        
        
    for row in soup.find('table').find_all('tr')[1:]:
        
        row_names.append(row.find('th').text)
        
        try:
            row_values.append(row.find('td').text)
            
        except AttributeError:
            row_values.append(row.find('td').find('a').text)
    
    return pd.DataFrame(row_values, row_names).T
    
    

In [69]:
get_full_bank_info('BUINBGSFXXX', proxies[0])

,Swift code (8 characters),Branch name,Branch address,Branch code,Bank name,City,Country
0,BUINBGSF,ALLIANZ BANK BULGARIA AD,79 MARIA LOUISA BOULEVARD,XXX,\nALLIANZ BANK BULGARIA AD\n,SOFIA,\nBulgaria\n


In [115]:
store_info = []

for idx, swift in enumerate(banks_info_df.swift.iloc[1100:], start=1100):
    
    if idx % 10 == 0:
        proxy = proxies[randint(0, 99)]
        
    store_info.append(get_full_bank_info(swift, proxy))
        
    if idx % 100 == 0:
        print(f'processed {idx} href(s)')
        
    
banks_full_info = pd.concat(store_info)

processed 1100 href(s)
processed 1200 href(s)
processed 1300 href(s)
processed 1400 href(s)
processed 1500 href(s)
processed 1600 href(s)
processed 1700 href(s)
processed 1800 href(s)
processed 1900 href(s)
processed 2000 href(s)
processed 2100 href(s)
processed 2200 href(s)
processed 2300 href(s)
processed 2400 href(s)
processed 2500 href(s)
processed 2600 href(s)
processed 2700 href(s)
processed 2800 href(s)
processed 2900 href(s)
processed 3000 href(s)
processed 3100 href(s)
processed 3200 href(s)
processed 3300 href(s)
processed 3400 href(s)
processed 3500 href(s)
processed 3600 href(s)
processed 3700 href(s)
processed 3800 href(s)
processed 3900 href(s)
processed 4000 href(s)
processed 4100 href(s)
processed 4200 href(s)
processed 4300 href(s)
processed 4400 href(s)
processed 4500 href(s)
processed 4600 href(s)
processed 4700 href(s)
processed 4800 href(s)
processed 4900 href(s)
processed 5000 href(s)
processed 5100 href(s)
processed 5200 href(s)
processed 5300 href(s)
processed 5

In [243]:
banks_full_info = pd.concat(store_info).drop_duplicates().reset_index(drop=True)

In [244]:
banks_full_info.columns

Index(['Swift code (8 characters)', 'Branch name', 'Branch address',
       'Branch code', 'Bank name', 'City', 'Country'],
      dtype='object')

In [245]:
banks_full_info.loc[:, ['Bank name', 'Country']] = \
    banks_full_info.loc[:, ['Bank name', 'Country']].applymap(lambda x: x.strip())

banks_full_info.loc[:, 'swift_full'] = \
    banks_full_info['Swift code (8 characters)'] + banks_full_info['Branch code']

banks_full_info = (banks_full_info.rename({'Swift code (8 characters)': 'swift_short',
                                          'Branch name': 'branch_name',
                                          'Branch address': 'branch_address',
                                          'Branch code': 'branch_code',
                                          'Bank name': 'bank_name',
                                          'City': 'city',
                                          'Country': 'country'
                                         }, axis=1)
                   .loc[:, ['country', 'city', 'bank_name', 'branch_name',
                            'branch_code', 'branch_address', 'swift_short', 'swift_full']]
                  )

In [246]:
banks_full_info

,country,city,bank_name,branch_name,branch_code,branch_address,swift_short,swift_full
0,Bulgaria,SOFIA,ALLIANZ BANK BULGARIA AD,ALLIANZ BANK BULGARIA AD,XXX,79 MARIA LOUISA BOULEVARD,BUINBGSF,BUINBGSFXXX
1,Bulgaria,SOFIA,BNP PARIBAS S.A.-SOFIA BRANCH,BNP PARIBAS S.A.-SOFIA BRANCH,XXX,"2, TZAR OSVOBODITEL BLVD",BNPABGSX,BNPABGSXXXX
2,Bulgaria,SOFIA,BORICA AD,BORICA AD,XXX,"41, TZAR BORIS III BLVD.",BSBGBGSF,BSBGBGSFXXX
3,Bulgaria,SOFIA,BORICA AD,BORICA AD,IOP,,BSBGBGSF,BSBGBGSFIOP
4,Bulgaria,SOFIA,BULGARIAN DEVELOPMENT BANK AD,BULGARIAN DEVELOPMENT BANK AD,XXX,"1, DYAKON IGNATIY STREET",NASBBGSF,NASBBGSFXXX
...,...,...,...,...,...,...,...,...
5953,United States,SALT LAKE CITY,ZIONS BANCORPORATION N.A.,ZIONS BANCORPORATION N.A.,SLC,"1 SOUTH MAIN STREET, FLOOR 4",ZFNBUS55,ZFNBUS55SLC
5954,United States,SEATTLE,ZIONS BANCORPORATION N.A.,ZIONS BANCORPORATION N.A.,SEA,601 UNION STREET,ZFNBUS55,ZFNBUS55SEA
5955,United States,SCHAUMBURG,ZURICH AMERICAN INSURANCE COMPANY,ZURICH AMERICAN INSURANCE COMPANY,FAR,1400 AMERICAN LANE,ZURIUS44,ZURIUS44FAR
5956,United States,SCHAUMBURG,ZURICH AMERICAN INSURANCE COMPANY,ZURICH AMERICAN INSURANCE COMPANY,XXX,1299 ZURICH WAY,ZURIUS44,ZURIUS44XXX


In [157]:
# banks_full_info.to_csv('banks_full_info.csv', index=False)

In [100]:
# banks_full_info.to_excel('banks_full_info.xlsx', index=False)

### Merging all banks database with detail table

Some of banks that were available on the website didn't appear in the resulting file. Check all missing swifts once again with requests by BIC.

In [17]:
banks_full_info = pd.read_csv('banks_full_info.csv')

In [18]:
banks_full_info[banks_full_info.swift_full.str.contains('ZABAHR2X')]

,country,city,bank_name,branch_name,branch_code,branch_address,swift_short,swift_full
72,Croatia,OSIJEK,ZAGREBACKA BANKA DD,ZAGREBACKA BANKA DD,OSI,TRG SLOBODE 4,ZABAHR2X,ZABAHR2XOSI
73,Croatia,POREC,ZAGREBACKA BANKA DD,ZAGREBACKA BANKA DD,POR,OBALA M. TITA 8B,ZABAHR2X,ZABAHR2XPOR
74,Croatia,PULA,ZAGREBACKA BANKA DD,ZAGREBACKA BANKA DD,PUL,LAGINJINA 1,ZABAHR2X,ZABAHR2XPUL
75,Croatia,MAKARSKA,ZAGREBACKA BANKA DD,ZAGREBACKA BANKA DD,MAK,JADRANSKA 1A,ZABAHR2X,ZABAHR2XMAK
76,Croatia,NOVA GRADISKA,ZAGREBACKA BANKA DD,ZAGREBACKA BANKA DD,NOV,TRG KRALJA TOMISLAVA 13,ZABAHR2X,ZABAHR2XNOV


In [253]:
detail_with_swift_addr = \
    (detail_with_swifts.merge(banks_full_info.query('branch_code == "XXX"')
                              .loc[:, ['bank_name', 'branch_name', 'city',
                                       'branch_code', 'branch_address', 'swift_short', 'swift_full']],
                             on='swift_short',
                             how='left'
                            )
    )

In [256]:
detail_with_swift_addr.to_csv('detail_with_swift_with_missing_addr.csv')

### Getting missing main branches addresses

In [38]:
detail_with_swift_addr = pd.read_csv('detail_with_swift_with_missing_addr.csv', index_col=0)

In [43]:
missing_address = \
    detail_with_swift_addr[detail_with_swift_addr.city.isnull()]

In [44]:
detail_with_swift_addr

,Region,Bank Country,# of banks by country,Bank Name,Cash Type,Currency,BIC,# of accounts,swift_short,bank_name,branch_name,city,branch_code,branch_address,swift_full
0,EMEA,Bulgaria,1.0,UniCredit Bulbank AD,Merchant Cash,BGN,UNCRBGSF,1,UNCRBGSF,NaN,NaN,NaN,NaN,NaN,NaN
1,EMEA,Croatia,1.0,Zagrebacka Banka,Merchant Cash,HRK,ZABAHR2X,2,ZABAHR2X,NaN,NaN,NaN,NaN,NaN,NaN
2,EMEA,Czech Republic,9.0,"Česká spořitelna, a.s.",Merchant Cash,CZK,GIBACZPX,9,GIBACZPX,CESKA SPORITELNA A.S.,CESKA SPORITELNA A.S.,PRAGUE,XXX,OLBRACHTOVA 62,GIBACZPXXXX
3,EMEA,Czech Republic,NaN,"Československá obchodní banka, a.s.",Merchant Cash,CZK,CEKOCZPP,3,CEKOCZPP,"CESKOSLOVENSKA OBCHODNI BANKA, A.S.","CESKOSLOVENSKA OBCHODNI BANKA, A.S.",PRAGUE,XXX,RADLICKA 333/150,CEKOCZPPXXX
4,EMEA,Czech Republic,NaN,"Fio banka, a.s.",Merchant Cash,CZK,FIOBCZPP,2,FIOBCZPP,"FIO BANKA, A.S.","FIO BANKA, A.S.",PRAGUE,XXX,"MILLENNIUM PLAZA, FLOOR 1, V CELNICI 1028/10",FIOBCZPPXXX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,LATAM,Spain,2.0,Banco Santander S.A.,Trade Cash,EUR,NaN,1,BSCHESMM,NaN,NaN,NaN,NaN,NaN,NaN
250,LATAM,Spain,NaN,Banco Santander S.A.,Trade Cash,USD,NaN,1,BSCHESMM,NaN,NaN,NaN,NaN,NaN,NaN
251,LATAM,Spain,NaN,Pershing LLC,Trade Cash,USD,NaN,1,PRSHUS33,NaN,NaN,NaN,NaN,NaN,NaN
252,LATAM,United States,2.0,"Citibank, N.A.",Trade Cash,USD,NaN,1,CITIUS33,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
missing_address.swift_short.unique()

array(['UNCRBGSF', 'ZABAHR2X', nan, 'POALILIT', 'KCBLKENX', 'INGBNL2A',
       'UNAFNGLA', 'EBOSPLPW', 'BREXPLPW', 'BPKOPLPW', 'BTRLRO22',
       'VTBRRUM2', 'ABSAZAJJ', 'SBZAZAJJ', 'ASYATRIS', 'FNNBTRIS',
       'TEBUTRIS', 'TGBATRIS', 'TRHBTR2A', 'TVBATR2A', 'YAPITRIS',
       'BBOGCOBB', 'COLOCOBM', 'BNMXMXMM', 'BSCHESMM', 'PRSHUS33\xa0',
       'CITIUS33'], dtype=object)

In [70]:
store_missing = []

for swift in missing_address.swift_short.unique():
    
    if isinstance(swift, str):
        
        try:
            store_missing.append(get_full_bank_info(swift + 'XXX', proxies[1]))
        except AttributeError:
            store_missing.append(pd.DataFrame({}))
    
    
retrieved_missing = pd.concat(store_missing)

In [73]:
retrieved_missing

,Swift code (8 characters),Branch name,Branch address,Branch code,Bank name,City,Country
0,UNCRBGSF,UNICREDIT BULBANK AD,"7, Sveta Nedelya Sq.",XXX,\nUNICREDIT BULBANK AD\n,SOFIA,\nBulgaria\n
0,ZABAHR2X,ZAGREBACKA BANKA DD,TRG BANA JOSIPA JELACICA 10,XXX,\nZAGREBACKA BANKA DD\n,ZAGREB,\nCroatia\n
0,POALILIT,BANK HAPOALIM B.M.,50 ROTHSCHILD BOULEVARD,XXX,\nBANK HAPOALIM B.M.\n,TEL AVIV,\nIsrael\n
0,KCBLKENX,KCB BANK KENYA LIMITED,"KENCOM HOUSE, FLOOR 7, MOI AVENUE",XXX,\nKCB BANK KENYA LIMITED\n,NAIROBI,\nKenya\n
0,INGBNL2A,ING BANK N.V.,BIJLMERDREEF 106,XXX,\nING BANK N.V.\n,AMSTERDAM,\nNetherlands\n
0,UNAFNGLA,UNITED BANK FOR AFRICA PLC,"UBA HOUSE, FLOOR 8, 57 MARINA",XXX,\nUNITED BANK FOR AFRICA PLC\n,LAGOS,\nNigeria\n
0,EBOSPLPW,BANK OCHRONY SRODOWISKA S.A.,UL. ZELAZNA 32,XXX,\nBANK OCHRONY SRODOWISKA S.A.\n,WARSZAWA,\nPoland\n
0,BREXPLPW,MBANK S.A. (FORMERLY BRE BANK S.A.),UL. SENATORSKA 18,XXX,\nMBANK S.A. (FORMERLY BRE BANK S.A.)\n,WARSZAWA,\nPoland\n
0,BPKOPLPW,PKO BANK POLSKI S.A.,UL. PULAWSKA 15,XXX,\nPKO BANK POLSKI S.A.\n,WARSZAWA,\nPoland\n
0,BTRLRO22,BANCA TRANSILVANIA S.A.,BARITIU STREET 8 G,XXX,\nBANCA TRANSILVANIA S.A.\n,CLUJ-NAPOCA,\nRomania\n


In [80]:
retrieved_missing.loc[:, ['Bank name', 'Country']] = \
    retrieved_missing.loc[:, ['Bank name', 'Country']].applymap(lambda x: x.strip())

retrieved_missing['swift_full'] = retrieved_missing['Swift code (8 characters)'] + retrieved_missing['Branch code']

retrieved_missing = \
    (retrieved_missing.rename({'Swift code (8 characters)': 'swift_short',
                               'Branch name': 'branch_name',
                               'Branch address': 'branch_address',
                               'Branch code': 'branch_code',
                               'Bank name': 'bank_name',
                               'City': 'city',
                               'Country': 'country'
                              }, axis=1)
     .loc[:, ['country', 'city', 'bank_name', 'branch_name',
              'branch_code', 'branch_address', 'swift_short', 'swift_full']]
    )

In [90]:
retrieved_missing = \
    (missing_address
     .drop(['bank_name', 'city', 'branch_name', 'branch_code', 'branch_address', 'swift_full'], axis=1)
     .merge(retrieved_missing, on='swift_short', how='left')
    )

In [97]:
detail_handled_missing = \
    (pd.concat([detail_with_swift_addr[detail_with_swift_addr.city.notnull()],
               retrieved_missing])
     .reset_index(drop=True)
     .drop('country', axis=1)
    )

In [98]:
detail_handled_missing

,Region,Bank Country,# of banks by country,Bank Name,Cash Type,Currency,BIC,# of accounts,swift_short,bank_name,branch_name,city,branch_code,branch_address,swift_full
0,EMEA,Czech Republic,9.0,"Česká spořitelna, a.s.",Merchant Cash,CZK,GIBACZPX,9,GIBACZPX,CESKA SPORITELNA A.S.,CESKA SPORITELNA A.S.,PRAGUE,XXX,OLBRACHTOVA 62,GIBACZPXXXX
1,EMEA,Czech Republic,NaN,"Československá obchodní banka, a.s.",Merchant Cash,CZK,CEKOCZPP,3,CEKOCZPP,"CESKOSLOVENSKA OBCHODNI BANKA, A.S.","CESKOSLOVENSKA OBCHODNI BANKA, A.S.",PRAGUE,XXX,RADLICKA 333/150,CEKOCZPPXXX
2,EMEA,Czech Republic,NaN,"Fio banka, a.s.",Merchant Cash,CZK,FIOBCZPP,2,FIOBCZPP,"FIO BANKA, A.S.","FIO BANKA, A.S.",PRAGUE,XXX,"MILLENNIUM PLAZA, FLOOR 1, V CELNICI 1028/10",FIOBCZPPXXX
3,EMEA,Czech Republic,NaN,"Komerční banka, a.s.",Merchant Cash,CZK,KOMBCZPP,2,KOMBCZPP,KOMERCNI BANKA A.S.,KOMERCNI BANKA A.S.,PRAGUE,XXX,NA PRIKOPE 33,KOMBCZPPXXX
4,EMEA,Czech Republic,NaN,"Komerční banka, a.s.",Trade cash,CZK,KOMBCZPP,1,KOMBCZPP,KOMERCNI BANKA A.S.,KOMERCNI BANKA A.S.,PRAGUE,XXX,NA PRIKOPE 33,KOMBCZPPXXX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,LATAM,Mexico,NaN,"Banco Nacional de Mexico, S.A.",Trade Cash,MXN,NaN,1,BNMXMXMM,BANCO NACIONAL DE MEXICO S.A.,BANCO NACIONAL DE MEXICO S.A.,MEXICO,XXX,"ACTUARIO ROBERTO MEDELLIN 800, COL. SANTA FE",BNMXMXMMXXX
250,LATAM,Spain,2.0,Banco Santander S.A.,Trade Cash,EUR,NaN,1,BSCHESMM,BANCO SANTANDER S.A.,BANCO SANTANDER S.A.,MADRID,XXX,"CIUDAD GRUPO SANTANDER, EDIFICIO PAMPA, CAMINO...",BSCHESMMXXX
251,LATAM,Spain,NaN,Banco Santander S.A.,Trade Cash,USD,NaN,1,BSCHESMM,BANCO SANTANDER S.A.,BANCO SANTANDER S.A.,MADRID,XXX,"CIUDAD GRUPO SANTANDER, EDIFICIO PAMPA, CAMINO...",BSCHESMMXXX
252,LATAM,Spain,NaN,Pershing LLC,Trade Cash,USD,NaN,1,PRSHUS33,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
# detail_handled_missing.to_excel('detail_all_done.xlsx', index=False)

## Matching banks from web with summary by names (optional)

In [293]:
banks_info_df['name_from_wise'] = \
    (banks_info_df.name_from_wise
     .str.lower()
     .apply(lambda x: re.sub('[–,\.\(\)-]', '', x))
    )

In [294]:
banks_info_df

,swift,name_from_wise,bank_country
0,BUINBGSFXXX,allianz bank bulgaria ad,bulgaria
0,BNPABGSXXXX,bnp paribas sasofia branch,bulgaria
0,BSBGBGSFXXX,borica ad,bulgaria
1,BSBGBGSFIOP,borica ad,bulgaria
0,NASBBGSFXXX,bulgarian development bank ad,bulgaria
...,...,...,...
3,ZFNBUS55SLC,zions bancorporation na,united states
4,ZFNBUS55SEA,zions bancorporation na,united states
0,ZURIUS44FAR,zurich american insurance company,united states
1,ZURIUS44XXX,zurich american insurance company,united states


In [468]:
bic_data = bic_data.rename({'Region': 'region',
                            'Bank Country': 'bank_country',
                            '# of banks by country': 'n_banks_by_country',
                            '# of banks running': 'n_banks_running',
                            'Bank Name': 'bank_name',
                            '# of accounts': 'n_accounts'
                           }, axis = 1)

bic_data.bank_country = bic_data.bank_country.str.lower()

bic_data.bank_name = \
    (bic_data.bank_name
     .str.lower()
     .apply(lambda x: re.sub('[–,\.\(\)-]', '', x))
    )

In [469]:
bic_data.head()

,region,bank_country,n_banks_by_country,n_banks_running,bank_name,n_accounts
0,EMEA,bulgaria,1.0,1.0,unicredit bulbank ad,1
1,EMEA,croatia,1.0,2.0,zagrebacka banka,2
2,EMEA,czech republic,9.0,3.0,česká spořitelna as,9
3,EMEA,czech republic,NaN,4.0,československá obchodní banka as,3
4,EMEA,czech republic,NaN,5.0,fio banka as,2


In [280]:
def min_levenshtein_dist(banks_info_df, bank):
    
    max_swift_bank = ''
    max_swift_dist = float('inf')
    
    for bank_from_info_df, swift in zip(banks_info_df.name_from_wise, banks_info_df.swift):
        
        lev_dist = nltk.edit_distance(bank, bank_from_info_df)
        
        if lev_dist < max_swift_dist:
            max_swift_bank = bank_from_info_df
            max_swift_dist = lev_dist
            
    return pd.DataFrame({'bank_name': [bank],
                         'best_match': [max_swift_bank],
                         'letter_mismatch': [max_swift_dist],
                         'swift': [swift]
                        })



In [281]:
min_levenshtein_dist(banks_info_df, 'československá obchodní banka as')

,bank_name,best_match,letter_mismatch,swift
0,československá obchodní banka as,ceskoslovenska obchodni banka as,3,ZURIUS44XXX


In [296]:
store_similar = []

for country in bic_data.bank_country.unique():
    
    bic_data_country = bic_data.query('bank_country == @country')
    banks_info_country = banks_info_df.query('bank_country == @country')
    
    for bank in bic_data_country.bank_name:
        store_similar.append(min_levenshtein_dist(banks_info_country, bank))
    
    print(f'found similars for {country}')

matches = pd.concat(store_similar)

found similars for bulgaria
found similars for croatia
found similars for czech republic
found similars for united kingdom
found similars for hungary
found similars for israel
found similars for kenya
found similars for netherlands
found similars for nigeria
found similars for poland
found similars for romania
found similars for russia
found similars for slovakia
found similars for south africa
found similars for turkey
found similars for argentina
found similars for chile
found similars for colombia
found similars for mexico
found similars for panama
found similars for peru
found similars for spain
found similars for united states


In [297]:
matches.query('letter_mismatch > 3')

,bank_name,best_match,letter_mismatch,swift
0,mbank sa,air bank as,6,ZENGCZPPXXX
0,ppro payment services sa,hsbc fund services,13,ZURIGB2LEND
0,unicredit bank hungary,unicredit bank hungary zrt,4,BACXHUHBINM
0,bank leumi,bank of israel,9,UNBKILITINT
0,citibank na israel branch,union bank of israel ltd,13,UNBKILITINT
0,cellulant,hfc limited,9,VICMKENAXXX
0,kcb,hfc limited,10,VICMKENAXXX
0,equity bank,prime bank ltd,9,VICMKENAXXX
0,citibank nigeria ltd,citibank nigeria limited,4,ZEPCNGLAXXX
0,first city monument bank,first city monument bank limited,8,ZEPCNGLAXXX


In [331]:
banks_info_df.query('name_from_wise.str.contains("handlowy")')

,swift,name_from_wise,bank_country
0,CITIPLPXXXX,bank handlowy w warszawie sa,poland
1,CITIPLPXCCH,bank handlowy w warszawie sa,poland


### Perform match search with masking by separate words

In [470]:
def replace_letters(word):
    
    for old, new in {'č': 'c', 'á': 'a', 'ř': 'r', 'í': 'i'}.items():
        word = word.replace(old, new)
    
    return word

In [471]:
bic_data['bank_name'] = \
    bic_data.bank_name.apply(lambda x: replace_letters(x))

In [472]:
bic_data['bank_name_split'] = \
    bic_data.bank_name.str.split()

In [522]:
banks_info_df

,swift,name_from_wise,bank_country,swift_cut
0,BUINBGSFXXX,allianz bank bulgaria ad,bulgaria,BUINBGSF
0,BNPABGSXXXX,bnp paribas sasofia branch,bulgaria,BNPABGSX
0,BSBGBGSFXXX,borica ad,bulgaria,BSBGBGSF
1,BSBGBGSFIOP,borica ad,bulgaria,BSBGBGSF
0,NASBBGSFXXX,bulgarian development bank ad,bulgaria,NASBBGSF
...,...,...,...,...
3,ZFNBUS55SLC,zions bancorporation na,united states,ZFNBUS55
4,ZFNBUS55SEA,zions bancorporation na,united states,ZFNBUS55
0,ZURIUS44FAR,zurich american insurance company,united states,ZURIUS44
1,ZURIUS44XXX,zurich american insurance company,united states,ZURIUS44


In [478]:
banks_info_df['swift_cut'] = \
    banks_info_df.swift.apply(lambda x: x[:-3])

banks_no_dupl = banks_info_df.drop_duplicates(subset=['swift_cut'])

In [479]:
banks_no_dupl

,swift,name_from_wise,bank_country,swift_cut
0,BUINBGSFXXX,allianz bank bulgaria ad,bulgaria,BUINBGSF
0,BNPABGSXXXX,bnp paribas sasofia branch,bulgaria,BNPABGSX
0,BSBGBGSFXXX,borica ad,bulgaria,BSBGBGSF
0,NASBBGSFXXX,bulgarian development bank ad,bulgaria,NASBBGSF
0,BNBGBGSFXXX,bulgarian national bank,bulgaria,BNBGBGSF
...,...,...,...,...
0,XERXUS31XXX,xerox corporation,united states,XERXUS31
0,YIFTUS32XXX,yisrael family trust,united states,YIFTUS32
0,TRWAUS33XXX,zf automotive us inc,united states,TRWAUS33
0,ZFNBUS55LAX,zions bancorporation na,united states,ZFNBUS55


In [480]:
def check_levenshtein_for_result(result, name_lst):
    
    best_match = ''
    mismatch_cnt = float('inf')
    
    for name in result.name_from_wise:
        lev = nltk.edit_distance(name, ' '.join(name_lst))
        
        if lev < mismatch_cnt:
            mismatch_cnt = lev
            best_match = name
    
    res_df = result.query('name_from_wise == @best_match')
    res_df['name_from_xlsx'] = ' '.join(name_lst)
    res_df['mismatch_cnt'] = mismatch_cnt
    
    return res_df



In [523]:
def search_by_word(banks_info_df, country, name_lst):
    
    store_lev_checks = []
    
    for word in name_lst:
        
        word = f' {word} |^{word} | {word}$'
        
        banks_info_country = banks_info_df.query('bank_country == @country')
        result = banks_info_country.query('name_from_wise.str.contains(@word)')
        
        if result.shape[0] == 1:
            result['name_from_xlsx'] = ' '.join(name_lst)
            result['mismatch_cnt'] = 0
            return result
        
        elif result.shape[0] >= 2:
            
            lev_one_word = check_levenshtein_for_result(result, name_lst)
            store_lev_checks.append(lev_one_word)
    
    if store_lev_checks:
        mismatches = pd.concat(store_lev_checks)
        return mismatches.query('mismatch_cnt == mismatch_cnt.min()')
    
    return pd.DataFrame({'swift': [],
                         'name_from_wise': [],
                         'bank_country': [],
                         'swift_cut': [],
                         'name_from_xlsx': [],
                         'mismatch_cnt': []
                        })
      


In [510]:
search_by_word(banks_no_dupl, no_swift_1.bank_country.iloc[16], no_swift_1.bank_name_split.iloc[16])

<ipython-input-480-cfacb835e8b0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df['name_from_xlsx'] = ' '.join(name_lst)
<ipython-input-480-cfacb835e8b0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df['mismatch_cnt'] = mismatch_cnt


,swift,name_from_wise,bank_country,swift_cut,name_from_xlsx,mismatch_cnt
0,FNNBROBUXXX,credit europe bank romania sa,romania,FNNBROBU,paypal europe,22


In [527]:
store_search = []

for country in bic_data.bank_country.unique():
    
    for name in bic_data.query('bank_country == @country').bank_name_split:
        
        store_search.append(search_by_word(banks_no_dupl, country, name))
        
        
matches = pd.concat(store_search).drop_duplicates().drop('swift', axis=1)

<ipython-input-523-fbef0394c37f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['name_from_xlsx'] = ' '.join(name_lst)
<ipython-input-523-fbef0394c37f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['mismatch_cnt'] = 0
<ipython-input-480-cfacb835e8b0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

<ipython-input-480-cfacb835e8b0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df['mismatch_cnt'] = mismatch_cnt
<ipython-input-480-cfacb835e8b0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df['name_from_xlsx'] = ' '.join(name_lst)
<ipython-input-480-cfacb835e8b0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

<ipython-input-523-fbef0394c37f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['name_from_xlsx'] = ' '.join(name_lst)
<ipython-input-523-fbef0394c37f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['mismatch_cnt'] = 0
<ipython-input-480-cfacb835e8b0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

<ipython-input-480-cfacb835e8b0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df['name_from_xlsx'] = ' '.join(name_lst)
<ipython-input-480-cfacb835e8b0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df['mismatch_cnt'] = mismatch_cnt
<ipython-input-480-cfacb835e8b0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [528]:
matches.mismatch_cnt.value_counts().sort_index()

0.0     81
1.0      4
2.0      1
3.0      4
4.0      3
5.0      3
7.0      3
8.0      3
11.0     2
13.0     2
17.0     1
22.0     1
Name: mismatch_cnt, dtype: int64

In [532]:
matches = \
    (matches
     .rename({'swift_cut': 'swift_short', 'name_from_xlsx': 'initial_name'}, axis=1)
     .loc[:, ['bank_country', 'initial_name', 'name_from_wise', 'swift_short', 'mismatch_cnt']]
    )

In [535]:
banks_with_swifts = \
    bic_data.merge(matches.loc[:, ['name_from_wise', 'initial_name', 'swift_short', 'mismatch_cnt']],
                   left_on='bank_name',
                   right_on='initial_name',
                   how='left').drop(['initial_name', 'bank_name_split'], axis=1)

In [538]:
banks_with_swifts.to_excel('banks_with_swifts.xlsx', index=False)

In [383]:
no_swift_1 = bic_data.query('swift_cut.isnull()').drop(['name_from_wise', 'name_from_xlsx', 'swift_cut'], axis=1)

In [384]:
no_swift_1

,region,bank_country,n_banks_by_country,n_banks_running,bank_name,n_accounts,bank_name_split
7,EMEA,czech republic,NaN,8.0,moneta money bank as,2,"[moneta, money, bank, as]"
11,EMEA,united kingdom,1.0,12.0,ppro payment services sa,1,"[ppro, payment, services, sa]"
12,EMEA,hungary,2.0,13.0,otp bank nyrt,5,"[otp, bank, nyrt]"
17,EMEA,kenya,2.0,18.0,cellulant,1,[cellulant]
21,EMEA,netherlands,NaN,22.0,ing bank nv,29,"[ing, bank, nv]"
23,EMEA,nigeria,NaN,24.0,first bank of nigeria ltd,2,"[first, bank, of, nigeria, ltd]"
28,EMEA,nigeria,NaN,29.0,zenith bank plc,13,"[zenith, bank, plc]"
33,EMEA,poland,NaN,34.0,bank nowy bfg sa,7,"[bank, nowy, bfg, sa]"
36,EMEA,poland,NaN,37.0,bnp paribas bank polska sa,5,"[bnp, paribas, bank, polska, sa]"
40,EMEA,poland,NaN,41.0,ing bank śląski sa,20,"[ing, bank, śląski, sa]"


In [415]:
search_by_word(banks_no_dupl, no_swift_1.bank_country.iloc[2], no_swift_1.bank_name_split.iloc[2])

<ipython-input-411-c195483946ec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['name_from_xlsx'] = ' '.join(name_lst)


,swift,name_from_wise,bank_country,swift_cut,name_from_xlsx
0,OTPVHUHBXXX,otp bank plc,hungary,OTPVHUHB,otp bank nyrt
0,OTPJHUHBXXX,otp mortgagebank ltd,hungary,OTPJHUHB,otp bank nyrt


In [413]:
store_search = []

for country in no_swift_1.bank_country.unique():
    
    for name in no_swift_1.query('bank_country == @country').bank_name_split:
        
        store_search.append(search_by_word(banks_no_dupl, country, name))
        
        
matches_2 = pd.concat(store_search)

<ipython-input-411-c195483946ec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['name_from_xlsx'] = ' '.join(name_lst)


In [414]:
matches_2

,bank_name,best_match,letter_mismatch,swift,name_from_wise,bank_country,swift_cut,name_from_xlsx
0,moneta money bank as,moneta money bank as,0.0,ZURIUS44FAR,NaN,NaN,NaN,NaN
0,ppro payment services sa,easy payment services ltd,7.0,ZURIUS44FAR,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,OTPVHUHBXXX,otp bank plc,hungary,OTPVHUHB,otp bank nyrt
0,NaN,NaN,NaN,OTPJHUHBXXX,otp mortgagebank ltd,hungary,OTPJHUHB,otp bank nyrt
0,cellulant,bell bank,4.0,ZURIUS44FAR,NaN,NaN,NaN,NaN
0,ing bank nv,ing bank nv,0.0,ZURIUS44FAR,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,FBNINGLAGCS,first bank of nigeria ltd,nigeria,FBNINGLA,first bank of nigeria ltd
0,NaN,NaN,NaN,FCMBNGLAWEB,first city monument bank limited,nigeria,FCMBNGLA,first bank of nigeria ltd
0,NaN,NaN,NaN,ZEIBNGLA013,zenith bank plc,nigeria,ZEIBNGLA,zenith bank plc
0,NaN,NaN,NaN,ZEPCNGLAXXX,zenith pensions custodian limited,nigeria,ZEPCNGLA,zenith bank plc
